# Creating money spent per ad click

In [1]:
import pandas as pd

In [2]:
ad_clicks = pd.read_csv('ad-clicks.csv')
buy_clicks = pd.read_csv('buy-clicks.csv')

In [3]:
total_adclicks_per_user = ad_clicks.groupby(by='userId').count()['txId']
total_adclicks_per_user.head()

userId
1     44
8     10
9     37
10    19
12    46
Name: txId, dtype: int64

In [4]:
purchase_per_user = buy_clicks.groupby('userId').price.sum()
purchase_per_user.head()

userId
1      21.0
8      53.0
9      80.0
10     11.0
12    215.0
Name: price, dtype: float64

In [5]:
df_combined = pd.concat([total_adclicks_per_user, purchase_per_user], axis=1)
df_combined['purchase_per_click'] = df_combined.price/df_combined.txId
purchase_per_adclick_per_user  = df_combined.purchase_per_click

In [6]:
purchase_per_adclick_per_user.head()

userId
1     0.477273
8     5.300000
9     2.162162
10    0.578947
12    4.673913
Name: purchase_per_click, dtype: float64

# Creating average session length

In [7]:
user_session = pd.read_csv('user-session.csv')
user_session.head()

,timestamp,userSessionId,userId,teamId,assignmentId,sessionType,teamLevel,platformType
0,2016-05-26 14:47:30,5675,170,90,5423,start,1,iphone
1,2016-05-27 00:06:54,5675,170,90,5423,end,1,iphone
2,2016-05-26 14:43:09,5795,835,39,5300,start,1,android
3,2016-05-27 08:06:54,5795,835,39,5300,end,1,android
4,2016-05-26 14:47:20,5833,624,2,5529,start,1,iphone


In [8]:
user_session_start = user_session[user_session.sessionType=='start']
user_session_end = user_session[user_session.sessionType=='end']
user_session_depivot = pd.merge(user_session_start, user_session_end, 
                                left_on=['userId','userSessionId'], right_on=['userId','userSessionId'])
user_session_depivot.head()

,timestamp_x,userSessionId,userId,teamId_x,assignmentId_x,sessionType_x,teamLevel_x,platformType_x,timestamp_y,teamId_y,assignmentId_y,sessionType_y,teamLevel_y,platformType_y
0,2016-05-26 14:47:30,5675,170,90,5423,start,1,iphone,2016-05-27 00:06:54,90,5423,end,1,iphone
1,2016-05-26 14:43:09,5795,835,39,5300,start,1,android,2016-05-27 08:06:54,39,5300,end,1,android
2,2016-05-26 14:47:20,5833,624,2,5529,start,1,iphone,2016-05-27 23:06:54,2,5529,end,1,iphone
3,2016-05-26 14:37:33,5877,514,2,5528,start,1,windows,2016-05-27 23:06:54,2,5528,end,1,windows
4,2016-05-26 14:36:55,5902,2103,2,5527,start,1,iphone,2016-05-27 23:06:54,2,5527,end,1,iphone


In [9]:
from datetime import datetime as dt

In [10]:
def convert(x):
    y = dt.strptime(x, '%Y-%m-%d %H:%M:%S')
    return y

user_session_depivot['session_duration'] = list(map(lambda y,x: (convert(y)-convert(x)).total_seconds()/60,
                                                   user_session_depivot.timestamp_y,
                                                   user_session_depivot.timestamp_x))

In [11]:
session_duration = user_session_depivot.groupby('userId').session_duration.mean()

In [12]:
session_duration.head()

userId
0    2632.500000
1    3750.988095
2    3834.000000
6    4470.000000
8    2400.000000
Name: session_duration, dtype: float64

# Creating Hit Rate

In [13]:
game_clicks = pd.read_csv('game-clicks.csv')
game_clicks.head()

,timestamp,clickId,userId,userSessionId,isHit,teamId,teamLevel
0,2016-05-26 15:06:55,105,1038,5916,0,25,1
1,2016-05-26 15:07:09,154,1099,5898,0,44,1
2,2016-05-26 15:07:14,229,899,5757,0,71,1
3,2016-05-26 15:07:14,322,2197,5854,0,99,1
4,2016-05-26 15:07:20,22,1362,5739,0,13,1


In [14]:
hit_rate = game_clicks.groupby(by='userId').isHit.sum()/game_clicks.groupby(by='userId').isHit.count()

In [15]:
hit_rate.head()

userId
0    0.105535
1    0.134078
2    0.095238
6    0.105960
8    0.100000
Name: isHit, dtype: float64

# Combining everything

In [16]:
df_final = pd.concat([purchase_per_adclick_per_user, session_duration, hit_rate], axis=1)

In [17]:
df_final.fillna(value=0, inplace=True)
df_final.columns = ['purchase_per_adclick', 'avg_session_duration', 'hit_rate']
df_final.head(10)

,purchase_per_adclick,avg_session_duration,hit_rate
userId,,,
0,0.000000,2632.500000,0.105535
1,0.477273,3750.988095,0.134078
2,0.000000,3834.000000,0.095238
6,0.000000,4470.000000,0.105960
8,5.300000,2400.000000,0.100000
9,2.162162,3760.000000,0.122047
10,0.578947,3060.000000,0.109430
12,4.673913,3561.428571,0.130682
13,1.250000,4065.000000,0.102345


In [18]:
df_final.to_csv('Data_for_Clustering_3_Features.csv')

In [19]:
df_finalalal.count()

purchase_per_adclick    1193
avg_session_duration    1193
hit_rate                1193
dtype: int64